# Chains

Hello everyone! Here, we will explore the work behind chaining

Concepts covered in this notebook are:
1. [Basic Chaining](#1)
2. [Level 1 - Chaining is Passing](#2)
3. [Level 2 - Chaining is Runnable Sequence](#3)

## Load environment variables

In [1]:
from dotenv import load_dotenv

load_dotenv()

## By default, load_dotenv() will assign environment variables into os.environ, like following code:
## See environment variables in .env file
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] =  os.getenv('LANGCHAIN_API_KEY')
# os.environ["OPENAPI_KEY"] =  os.getenv('OPENAI_KEY')

True

Let's create language model instance, prompt template, and output parser

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

model = ChatOpenAI(model="gpt-3.5-turbo")

prompt_template = ChatPromptTemplate([
    ("system", "Translete the following into {language}"),
    ("user", "{text}")
])

parser = StrOutputParser()

# Basic Chaining

"Chaining" is operation to combine sequential processes — for example from prompt templates to output parsers— using the pipe (`|`) operator.

In [8]:
chain = prompt_template | model | parser

chain.invoke({"language": "indonesia", "text": "How are you?"})

'Apa kabar?'

## 2 - Chaining Under the Hood <div id="2"></div>

Chaining is an incredible operation in LangChain. While understanding it as a combination of sequential processes is a good start, mastering chaining requires a deeper understanding of how it works internally. I will explain this with three levels of abstraction.

### 2.1 -  Chaining is Passing Output From Previous to Next Component 

Here’s a breakdown of what happens during chaining:

<img src="../../images/1.png" width="600px"/>

- Invoke the Chain: Pass initial arguments (e.g., language and text) to the chain object.

- Pass to `prompt_template`: Convert the arguments into messages using the prompt_template.

- Pass to `model`: Generate a response from the model based on the formatted messages.

- Pass to `parser`: Convert the model's response into the final output format using the parser.

This sequential flow allows each component to process and transform data effectively.

### 2.2 - Chaining is Creating `RunnableSequence` containing `Runnable`s

Chain operation (`|`) in langchain is part of LangChain Expression Language (LCEL). 

When we create a chain, we are essentially creating an object called `RunnableSequence`. Each component in this sequence must implement the `Runnable` interface.

For example:

```python
chain = prompt_template | model | parser
```

In this code:
- `chain` is an instance of `RunnableSequence`.
- `prompt_template`, `model`, and `parser` are instances of components that implement the `Runnable` interface.

### `Runnable` Interface

Many LangChain components, including chat models, LLMs, output parsers, retrievers, and prompt templates, implement the `Runnable` interface. This interface includes two most important methods:

- **`invoke`**: Calls the chain on a single input.
- **`__or__`**: This is operator overloading, that enable Runnable to overwrite `|` (pipe) operator and turn it into forming RunnableSequence. This looks like:

```python
class Runnable:
    def __or__(self, other):
        # Handle chaining with the pipe operator
        return RunnableSequence(first=self, middle=[other], last=None)
```

For example:
```python
chain = component1 | component2

# is equivalent to
chain = component1.__or__(component2)

# Equivalent to
chain = RunnableSequence(first=component1, last=component2)
```

### `RunnableSequence` instances

RunnableSequence is a special type of `Runnable` that allows chaining multiple `Runnable`s together. It has three important attributes:

- `first`: The first `Runnable` in the sequence.
- `middle`: A list of `Runnable`s that follow the `first` `Runnable`.
- `last`: The last `Runnable` in the sequence.

RunnableSequence also has two most important methods:

- **`invoke`**: Calls the chain on a single input.
- **`__or__`**: override pipe (`|`) operator

For example:

```python
chain = component1 | component2 | component3

# Equivalent to

chain = component1.__or__(component2).__or__(component3)

# Equivalent to

chain = RunnableSequence(first=component1, last=component2).__or__(component3)

# Equivalent to

chain = RunnableSequence(first=component1, middle=[component2], last=component3)

```

You can learn more about the `Runnable` interface in the [Runnable Interface documentation](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface).

**Let's experiment with Runnable and RunnableSequence**

To make a function become `Runnable` we can pass it to `RunnableLambda` class

In [23]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence

# Create a Runnable functions
double = RunnableLambda(lambda x: 2*x)
square = RunnableLambda(lambda x: x**2)
tenth = RunnableLambda(lambda x : x / 10)

# Create a Runnable sequence using pipe operator
chain = double | square | tenth

result1 = chain.invoke(5)

print(f"Result from chain invocation: {result1}")

# Try to invoke sequence one by one and passing it
result2 = double.invoke(5)
result2 = square.invoke(result2)
result2 = tenth.invoke(result2)

print(f"Result from component invocation and passing: {result2}")

# ((2*5)^2)/10 = 100/10 = 10

Result from chain invocation: 10.0
Result from component invocation and passing: 10.0


In [26]:
chain = double | square | tenth

result1 = chain.invoke(5)

# equivalent to

chain = double.__or__(square).__or__(tenth)

result2 = chain.invoke(5)

# equivalent to

chain = RunnableSequence(first=double, middle=[square], last=tenth)

result3 = chain.invoke(5)

print(result1)
print(result2)
print(result3)

10.0
10.0
10.0


In [29]:
# Define prompt templates
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells jokes about {topic}."),
        ("human", "Tell me {joke_count} jokes."),
    ]
)

# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: model.invoke(x))
parse_output = RunnableLambda(lambda x: x.content)

# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

# Run the chain
response = chain.invoke({"topic": "animals", "joke_count": 3})

# Output
print(response)


1. Why did the bee get married? Because he found his honey!

2. What do you call a bear with no teeth? A gummy bear!

3. Why did the chicken join a band? Because it had the drumsticks!
